In [1]:
import numpy as np
import scipy.linalg as sla
from PIL import Image
from scipy import misc
import scipy.sparse as sps
import scipy.sparse.linalg as spla
import timeit
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd

from tqdm.auto import tqdm

In [2]:
def prepare_photo_aug2(df, dir):
    """INPUT
    df -> pandas DataFrame: empty df
    dir -> str: abs path to dir with photo
        
        OTPUT -> pandas:DataFrame: full df"""
        
    path = '/Users/habirova-rr/Downloads/НИС'+f'/{dir}'
    photos = os.listdir(path)
    print(len(photos))

    for i, photo in enumerate(tqdm(photos)):
        try:
            photo_pth = f'{path}/'+f'{photo}'
            img = 255 - cv2.imread(photo_pth, cv2.IMREAD_GRAYSCALE) # это матрица из интенсивностей серого цвета

            width = 224
            height = 224
            dim = (width, height)

            # resize image
            #resized = cv2.resize(img, dim, interpolation = cv2.INTER_LANCZOS4)
            data0 = np.reshape(img, 50176)
            data0 = pd.Series(data0)
            df = df.append(data0, ignore_index=True)
            
        except:
            print(f'ALARM error with {i, photo}')
    return df

In [3]:
df_z = pd.DataFrame(columns=[i for i in range(50176)])

In [4]:
p1 = '1_предупреждающие'  #predupr
p2 = '2_приоритета'       #prior
p3 = '3_запрещающие'      #zapret
p4 = '4_предписывающие'   #predpis
p5 = '5_особых предписаний'#os_predpis
p6 = '6_информационные'   #info
p7 = '7_сервиса'          #service
p8 = '8_доп информации(таблички)'#tables

In [5]:
df1 = prepare_photo_aug2(df_z, p1)

84


In [6]:
df2 = prepare_photo_aug2(df_z, p2)

35


In [7]:
df3 = prepare_photo_aug2(df_z, p3)

124


In [8]:
df4 = prepare_photo_aug2(df_z, p4)

37


In [9]:
df5 = prepare_photo_aug2(df_z, p5)

152


In [10]:
df6 = prepare_photo_aug2(df_z, p6)

81


In [11]:
df7 = prepare_photo_aug2(df_z, p7)

28


In [12]:
df8 = prepare_photo_aug2(df_z, p8)

148


In [13]:
df1['target'] = 'predupr'
df2['target'] = 'prior'
df3['target'] = 'zapret'
df4['target'] = 'predpis'
df5['target'] = 'os_predpis'
df6['target'] = 'info'
df7['target'] = 'service'
df8['target'] = 'tables'

In [14]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0, ignore_index=True)

In [15]:
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix

In [16]:
# Закодируем лейблы
LE = LabelEncoder()
df.target = LE.fit_transform(df.target)

In [17]:
# Разделим выборку
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.target, test_size=0.3, random_state=42)

### KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
n_neighbors = [3, 5, 7, 9]

In [20]:
def knn(n):
    knn_model = KNeighborsClassifier(n_neighbors=n)
    knn_model.fit(x_train, y_train)
    y_pred = knn_model.predict(x_test)
    acc_score = round(accuracy_score(y_pred, y_test), 3)
    print(f'KNeighborsClassifier: n_neighbors - {n}, accuracy_score = {acc_score}')    

In [21]:
for n in n_neighbors:
    knn(n)

KNeighborsClassifier: n_neighbors - 3, accuracy_score = 0.734
KNeighborsClassifier: n_neighbors - 5, accuracy_score = 0.725
KNeighborsClassifier: n_neighbors - 7, accuracy_score = 0.696
KNeighborsClassifier: n_neighbors - 9, accuracy_score = 0.657


#### SVM

In [22]:
from sklearn.svm import SVC

In [35]:
kernels = ['linear']

In [59]:
reg_parameters = [0.05, 1.0, 1.5, 2.0]

In [55]:
def svm(c):
    svm_model = SVC(kernel='linear', C=c)
    svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)
    acc_score = round(accuracy_score(y_pred, y_test), 3)
    print(f'SVM: kernel - linear, c - {c}, accuracy_score = {acc_score}')    

In [56]:
for c in ccc:
    svm(c)

SVM: kernel - linear, c - 0.5, accuracy_score = 0.831
SVM: kernel - linear, c - 1.0, accuracy_score = 0.831
SVM: kernel - linear, c - 1.5, accuracy_score = 0.831
SVM: kernel - linear, c - 2.0, accuracy_score = 0.831
